# Python para Data Science com Bancos de Dados

## SQLite

In [70]:
import sqlite3
import pandas as pd

In [71]:
# Conexão (cria o arquivo se não existir)
conn = sqlite3.connect("/content/vinhos.db")

In [72]:
# Leitura de tabela existente
df = pd.read_sql("SELECT * FROM vinhos", conn)

# Exibir primeiras linhas
df.tail()

,year,month,location,location_type,category,grape_type,volume_hl,value_usd
26635,2024,12,China,Country,Import,Wine,3574,12650.27
26636,2024,12,China,Country,Import,Sparkling,825,3844.02
26637,2024,12,China,Country,Export,Table,2115,6153.84
26638,2024,12,China,Country,Export,Wine,2107,9355.02
26639,2024,12,China,Country,Export,Sparkling,6813,23101.92


In [73]:
# Consulta filtrada
df_brasil = pd.read_sql("SELECT * FROM vinhos WHERE location_type='Brazil_State'", conn)
df_brasil.tail()

,year,month,location,location_type,category,grape_type,volume_hl,value_usd
19435,2024,12,TO,Brazil_State,Production,Wine,1365,4895.26
19436,2024,12,TO,Brazil_State,Production,Sparkling,4912,14925.51
19437,2024,12,TO,Brazil_State,Sales,Table,3391,9911.72
19438,2024,12,TO,Brazil_State,Sales,Wine,306,1154.84
19439,2024,12,TO,Brazil_State,Sales,Sparkling,1717,3201.36


In [74]:
# exemplo de leitura de uma tabela 'vinho'
# (se não existir, adapte o nome)
try:
    df = pd.read_sql("SELECT * FROM vinho LIMIT 5", conn)
    display(df)
except Exception as e:
    print("Ajuste o nome da tabela. Erro:", e)

Ajuste o nome da tabela. Erro: Execution failed on sql 'SELECT * FROM vinho LIMIT 5': no such table: vinho


In [75]:
conn.close() # Fecha a conexão

## SQLAlchemy

### Exemplo geral

In [76]:
#Instalar o SQLAlchemy no google colab !

!pip install SQLAlchemy

In [77]:
#Improtar a engine para utilizar o pacote
from sqlalchemy import create_engine

In [78]:
# Criar engine para conectar ao SQLite
engine = create_engine("sqlite:///vinhos.db")

In [79]:
# Ler tabela
df = pd.read_sql("vinhos", engine)
df.tail()

,year,month,location,location_type,category,grape_type,volume_hl,value_usd
26635,2024,12,China,Country,Import,Wine,3574,12650.27
26636,2024,12,China,Country,Import,Sparkling,825,3844.02
26637,2024,12,China,Country,Export,Table,2115,6153.84
26638,2024,12,China,Country,Export,Wine,2107,9355.02
26639,2024,12,China,Country,Export,Sparkling,6813,23101.92


In [80]:
# Escrever nova tabela
df_filtrado = df[df['location_type'] == 'Brazil_State']
df_filtrado.to_sql("vinho_brasil", engine, if_exists="replace", index=False)

# Confirmar gravação
pd.read_sql("vinho_brasil", engine).head()

,year,month,location,location_type,category,grape_type,volume_hl,value_usd
0,2015,1,AC,Brazil_State,Production,Table,910,1782.31
1,2015,1,AC,Brazil_State,Production,Wine,3822,11435.90
2,2015,1,AC,Brazil_State,Production,Sparkling,516,902.97
3,2015,1,AC,Brazil_State,Sales,Table,3494,10491.74
4,2015,1,AC,Brazil_State,Sales,Wine,180,279.26


In [81]:
# Ler tabela 'vinho' como DataFrame (se existir)
try:
    df = pd.read_sql("SELECT * FROM vinhos LIMIT 10", engine)
    display(df.head())
except Exception as e:
    print("Ajuste o nome da tabela. Erro:", e)

,year,month,location,location_type,category,grape_type,volume_hl,value_usd
0,2015,1,AC,Brazil_State,Production,Table,910,1782.31
1,2015,1,AC,Brazil_State,Production,Wine,3822,11435.90
2,2015,1,AC,Brazil_State,Production,Sparkling,516,902.97
3,2015,1,AC,Brazil_State,Sales,Table,3494,10491.74
4,2015,1,AC,Brazil_State,Sales,Wine,180,279.26


In [82]:
# Ex.: gravar um DataFrame de resultados
resumo = pd.DataFrame({
    "categoria": ["Tinto","Branco"],
    "preco_medio": [55.2, 49.8]
})
resumo.to_sql("resumo_precos", engine, if_exists="replace", index=False)

pd.read_sql("SELECT * FROM resumo_precos", engine)

,categoria,preco_medio
0,Tinto,55.2
1,Branco,49.8


## Principais usos do SQLAlchemy

1 - Importação

In [83]:
# Importa tudo pra nao ter erro !
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, select, insert, update, delete, func
import pandas as pd

2 - Criando engine

In [84]:
engine = create_engine("sqlite:///vinho.db", echo=False)

metadata = MetaData()
vinho = Table(
    "vinho",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("pais", String),
    Column("categoria", String),
    Column("preco", Float),
)

# Caso a tabela ainda não exista:
metadata.create_all(engine)

3 - Insert

In [85]:
# Um registro
stmt = insert(vinho).values(pais="Portugal", categoria="Tinto", preco=89.0)
with engine.begin() as conn:
    conn.execute(stmt)

In [86]:
# Vários registros (bulk)
novos = [
    {"pais": "Itália", "categoria": "Branco", "preco": 75.5},
    {"pais": "Brasil", "categoria": "Rosé",  "preco": 45.9},
]
with engine.begin() as conn:
    conn.execute(insert(vinho), novos)

4 - Select

In [87]:
# Selecionar todas as colunas
stmt = select(vinho)                # SELECT * FROM vinho
with engine.connect() as conn:
    rows = conn.execute(stmt).fetchall()
print(rows[:5])

[(1, 'Portugal', 'Tinto', 89.0), (2, 'Itália', 'Branco', 75.5), (3, 'Brasil', 'Rosé', 45.9)]


In [88]:
# Filtrar + ordenar + limitar
stmt = (
    select(vinho.c.pais, vinho.c.preco)
    .where(vinho.c.pais == "Brasil")
    .order_by(vinho.c.preco.desc())
    .limit(5)
)
print(pd.read_sql(stmt, engine))

     pais  preco
0  Brasil   45.9


5 - Update

In [89]:
stmt = (
    update(vinho)
    .where(vinho.c.pais == "Argentina")
    .values(preco=vinho.c.preco * 1.1)  # aumento de 10 %
)
with engine.begin() as conn:
    conn.execute(stmt)

6 - Delete

In [90]:
stmt = delete(vinho).where(vinho.c.preco < 20)
with engine.begin() as conn:
    conn.execute(stmt)